In [42]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from collections import Counter

base_url = r'/home/amit/DataScienceProject/PredictDroughts/'
URL = r'https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_area'

In [43]:
dfDrought = pd.read_csv(base_url + 'files/drought.csv', index_col=0)
dfDrought

,State,Postal code,Fips,Week,None,D0-D4,D1-D4,D2-D4,D3-D4,D4,DSCI,LEVEL
0,Alabama,AL,1,2000-01-04,0.00,100.00,47.98,16.35,0.00,0.0,164,1
1,Alabama,AL,1,2000-01-11,0.00,100.00,76.88,4.24,0.00,0.0,181,1
2,Alabama,AL,1,2000-01-18,0.00,100.00,75.71,4.24,0.00,0.0,180,1
3,Alabama,AL,1,2000-01-25,0.00,100.00,75.67,4.79,0.00,0.0,180,1
4,Alabama,AL,1,2000-02-01,0.00,100.00,50.07,0.17,0.00,0.0,150,1
...,...,...,...,...,...,...,...,...,...,...,...,...
58235,Wyoming,WY,56,2021-05-18,10.89,89.11,70.75,27.15,2.01,0.0,189,1
58236,Wyoming,WY,56,2021-05-25,10.89,89.11,66.56,24.87,1.98,0.0,183,1
58237,Wyoming,WY,56,2021-06-01,10.89,89.11,68.06,24.28,1.98,0.0,183,1
58238,Wyoming,WY,56,2021-06-08,9.76,90.24,72.28,27.12,6.32,0.0,196,1


In [44]:
dfMeteorological = pd.read_csv(base_url + 'files/meteorological.csv', index_col=0)
dfMeteorological

,week,Longitude,Latitude,PRECTOT,WS10M_MIN,QV2M,T2M_RANGE,WS10M,T2M,WS50M_MIN,...,WS50M,TS,WS50M_RANGE,WS50M_MAX,WS10M_MAX,WS10M_RANGE,PS,T2MDEW,T2M_MIN,T2MWET
0,20000104,-86.902298,32.318231,12.35,2.14,6.84,15.39,3.80,12.27,3.55,...,6.75,12.18,5.24,8.80,5.44,3.30,100.72,6.95,2.85,6.97
1,20000111,-86.902298,32.318231,0.97,1.18,7.09,13.70,1.92,11.76,2.08,...,4.03,11.15,4.09,6.18,2.51,1.33,100.83,8.89,5.91,8.89
2,20000118,-86.902298,32.318231,1.80,1.59,9.91,6.50,2.54,15.00,3.54,...,5.14,14.94,2.75,6.29,3.87,2.28,100.82,13.94,11.96,13.94
3,20000125,-86.902298,32.318231,0.00,2.20,2.15,10.89,3.18,0.11,3.89,...,5.58,0.24,3.71,7.61,4.35,2.16,100.53,-7.34,-4.39,-7.14
4,20000201,-86.902298,32.318231,0.01,0.73,3.38,15.38,1.89,2.35,0.92,...,3.91,2.15,5.12,6.04,2.41,1.69,101.43,-1.58,-3.99,-1.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58235,20210518,-107.290284,43.075968,0.00,0.47,6.54,17.38,2.39,16.42,0.72,...,3.57,17.56,6.01,6.73,3.36,2.89,80.75,4.49,6.61,10.45
58236,20210525,-107.290284,43.075968,0.11,0.95,5.07,16.60,4.37,11.96,1.69,...,5.89,13.43,7.49,9.18,6.45,5.50,81.35,1.03,3.02,6.50
58237,20210601,-107.290284,43.075968,1.19,1.75,7.33,12.02,3.18,16.19,2.93,...,4.55,17.39,3.61,6.54,4.33,2.58,81.70,6.35,9.74,11.27
58238,20210608,-107.290284,43.075968,0.00,0.49,6.30,17.62,4.97,20.90,1.54,...,6.99,21.32,13.17,14.71,11.86,11.36,80.92,3.91,12.01,12.41


In [45]:
dfMeteorological.drop(columns='week', inplace=True)
dfMeteorological

,Longitude,Latitude,PRECTOT,WS10M_MIN,QV2M,T2M_RANGE,WS10M,T2M,WS50M_MIN,T2M_MAX,WS50M,TS,WS50M_RANGE,WS50M_MAX,WS10M_MAX,WS10M_RANGE,PS,T2MDEW,T2M_MIN,T2MWET
0,-86.902298,32.318231,12.35,2.14,6.84,15.39,3.80,12.27,3.55,18.24,6.75,12.18,5.24,8.80,5.44,3.30,100.72,6.95,2.85,6.97
1,-86.902298,32.318231,0.97,1.18,7.09,13.70,1.92,11.76,2.08,19.61,4.03,11.15,4.09,6.18,2.51,1.33,100.83,8.89,5.91,8.89
2,-86.902298,32.318231,1.80,1.59,9.91,6.50,2.54,15.00,3.54,18.45,5.14,14.94,2.75,6.29,3.87,2.28,100.82,13.94,11.96,13.94
3,-86.902298,32.318231,0.00,2.20,2.15,10.89,3.18,0.11,3.89,6.50,5.58,0.24,3.71,7.61,4.35,2.16,100.53,-7.34,-4.39,-7.14
4,-86.902298,32.318231,0.01,0.73,3.38,15.38,1.89,2.35,0.92,11.38,3.91,2.15,5.12,6.04,2.41,1.69,101.43,-1.58,-3.99,-1.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58235,-107.290284,43.075968,0.00,0.47,6.54,17.38,2.39,16.42,0.72,23.99,3.57,17.56,6.01,6.73,3.36,2.89,80.75,4.49,6.61,10.45
58236,-107.290284,43.075968,0.11,0.95,5.07,16.60,4.37,11.96,1.69,19.62,5.89,13.43,7.49,9.18,6.45,5.50,81.35,1.03,3.02,6.50
58237,-107.290284,43.075968,1.19,1.75,7.33,12.02,3.18,16.19,2.93,21.76,4.55,17.39,3.61,6.54,4.33,2.58,81.70,6.35,9.74,11.27
58238,-107.290284,43.075968,0.00,0.49,6.30,17.62,4.97,20.90,1.54,29.63,6.99,21.32,13.17,14.71,11.86,11.36,80.92,3.91,12.01,12.41


In [46]:
dfMeteorological['Postal Code'] = dfDrought['Postal code']
dfMeteorological['Week'] = dfDrought['Week']
dfMeteorological['LEVEL'] = dfDrought['LEVEL']
dfMeteorological['State'] = dfDrought['State']
dfMeteorological['Fips'] = dfDrought['Fips']
dfMeteorological

,Longitude,Latitude,PRECTOT,WS10M_MIN,QV2M,T2M_RANGE,WS10M,T2M,WS50M_MIN,T2M_MAX,...,WS10M_RANGE,PS,T2MDEW,T2M_MIN,T2MWET,Postal Code,Week,LEVEL,State,Fips
0,-86.902298,32.318231,12.35,2.14,6.84,15.39,3.80,12.27,3.55,18.24,...,3.30,100.72,6.95,2.85,6.97,AL,2000-01-04,1,Alabama,1
1,-86.902298,32.318231,0.97,1.18,7.09,13.70,1.92,11.76,2.08,19.61,...,1.33,100.83,8.89,5.91,8.89,AL,2000-01-11,1,Alabama,1
2,-86.902298,32.318231,1.80,1.59,9.91,6.50,2.54,15.00,3.54,18.45,...,2.28,100.82,13.94,11.96,13.94,AL,2000-01-18,1,Alabama,1
3,-86.902298,32.318231,0.00,2.20,2.15,10.89,3.18,0.11,3.89,6.50,...,2.16,100.53,-7.34,-4.39,-7.14,AL,2000-01-25,1,Alabama,1
4,-86.902298,32.318231,0.01,0.73,3.38,15.38,1.89,2.35,0.92,11.38,...,1.69,101.43,-1.58,-3.99,-1.55,AL,2000-02-01,1,Alabama,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58235,-107.290284,43.075968,0.00,0.47,6.54,17.38,2.39,16.42,0.72,23.99,...,2.89,80.75,4.49,6.61,10.45,WY,2021-05-18,1,Wyoming,56
58236,-107.290284,43.075968,0.11,0.95,5.07,16.60,4.37,11.96,1.69,19.62,...,5.50,81.35,1.03,3.02,6.50,WY,2021-05-25,1,Wyoming,56
58237,-107.290284,43.075968,1.19,1.75,7.33,12.02,3.18,16.19,2.93,21.76,...,2.58,81.70,6.35,9.74,11.27,WY,2021-06-01,1,Wyoming,56
58238,-107.290284,43.075968,0.00,0.49,6.30,17.62,4.97,20.90,1.54,29.63,...,11.36,80.92,3.91,12.01,12.41,WY,2021-06-08,1,Wyoming,56


In [47]:
dfMerge = dfMeteorological[['Week', 'State','Postal Code', 'Fips', 'Longitude', 'Latitude', 'PRECTOT' 
                                ,'WS10M_MIN' ,'QV2M','T2M_RANGE' ,'WS10M' ,'T2M' ,'WS50M_MIN' ,'T2M_MAX' 
                                ,'WS50M' ,'TS' ,'WS50M_RANGE' ,'WS50M_MAX' ,'WS10M_MAX' 
                                ,'WS10M_RANGE' ,'PS' ,'T2MDEW', 'T2M_MIN', 'T2MWET', 'LEVEL']]
dfMerge

,Week,State,Postal Code,Fips,Longitude,Latitude,PRECTOT,WS10M_MIN,QV2M,T2M_RANGE,...,TS,WS50M_RANGE,WS50M_MAX,WS10M_MAX,WS10M_RANGE,PS,T2MDEW,T2M_MIN,T2MWET,LEVEL
0,2000-01-04,Alabama,AL,1,-86.902298,32.318231,12.35,2.14,6.84,15.39,...,12.18,5.24,8.80,5.44,3.30,100.72,6.95,2.85,6.97,1
1,2000-01-11,Alabama,AL,1,-86.902298,32.318231,0.97,1.18,7.09,13.70,...,11.15,4.09,6.18,2.51,1.33,100.83,8.89,5.91,8.89,1
2,2000-01-18,Alabama,AL,1,-86.902298,32.318231,1.80,1.59,9.91,6.50,...,14.94,2.75,6.29,3.87,2.28,100.82,13.94,11.96,13.94,1
3,2000-01-25,Alabama,AL,1,-86.902298,32.318231,0.00,2.20,2.15,10.89,...,0.24,3.71,7.61,4.35,2.16,100.53,-7.34,-4.39,-7.14,1
4,2000-02-01,Alabama,AL,1,-86.902298,32.318231,0.01,0.73,3.38,15.38,...,2.15,5.12,6.04,2.41,1.69,101.43,-1.58,-3.99,-1.55,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58235,2021-05-18,Wyoming,WY,56,-107.290284,43.075968,0.00,0.47,6.54,17.38,...,17.56,6.01,6.73,3.36,2.89,80.75,4.49,6.61,10.45,1
58236,2021-05-25,Wyoming,WY,56,-107.290284,43.075968,0.11,0.95,5.07,16.60,...,13.43,7.49,9.18,6.45,5.50,81.35,1.03,3.02,6.50,1
58237,2021-06-01,Wyoming,WY,56,-107.290284,43.075968,1.19,1.75,7.33,12.02,...,17.39,3.61,6.54,4.33,2.58,81.70,6.35,9.74,11.27,1
58238,2021-06-08,Wyoming,WY,56,-107.290284,43.075968,0.00,0.49,6.30,17.62,...,21.32,13.17,14.71,11.86,11.36,80.92,3.91,12.01,12.41,1


In [48]:
def getAlandArea():
    response = requests.get(URL)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table')
    area_land_df = pd.read_html(str(table))[0]
    area_land_df = area_land_df.drop(columns=['Rank', '% land', '% water'], level=1)
    area_land_df = area_land_df.drop(area_land_df.index[[55, 56, 57, 58, 59]])
    area_land_df = area_land_df.sort_values([('State', 'State')], ascending=True)
    area_land_df.reset_index(drop=True, inplace=True)
    area_land_df = area_land_df.drop(area_land_df.index[[2, 35, 47]])
    area_land_df.reset_index(drop=True, inplace=True)
    area_land_df = area_land_df.drop(columns='Total area[2]', level=0)
    aland_sqmi_list = [float(x) * 0.62 for x in area_land_df['Land area[2]']['sq mi']] * len(dfMerge.loc[dfMerge['State'] == 'Alabama'])
    states_list = [x for x in area_land_df['State']['State']] * len(dfMerge.loc[dfMerge['State'] == 'Alabama'])

    data = {
        'State': states_list,
        'Aland_SQMI': aland_sqmi_list
    }

    df = pd.DataFrame(data)
    return df

aland_sqmi_df = getAlandArea()
aland_sqmi_df = aland_sqmi_df.sort_values('State').reset_index(drop=True)
aland_sqmi_df

,State,Aland_SQMI
0,Alabama,31400.1046
1,Alabama,31400.1046
2,Alabama,31400.1046
3,Alabama,31400.1046
4,Alabama,31400.1046
...,...,...
58235,Wyoming,60197.7468
58236,Wyoming,60197.7468
58237,Wyoming,60197.7468
58238,Wyoming,60197.7468


In [49]:
dfMerge.insert(len(dfMerge.columns) - 1, 'Aland_SQMI', aland_sqmi_df['Aland_SQMI'], None)
dfMerge

,Week,State,Postal Code,Fips,Longitude,Latitude,PRECTOT,WS10M_MIN,QV2M,T2M_RANGE,...,WS50M_RANGE,WS50M_MAX,WS10M_MAX,WS10M_RANGE,PS,T2MDEW,T2M_MIN,T2MWET,Aland_SQMI,LEVEL
0,2000-01-04,Alabama,AL,1,-86.902298,32.318231,12.35,2.14,6.84,15.39,...,5.24,8.80,5.44,3.30,100.72,6.95,2.85,6.97,31400.1046,1
1,2000-01-11,Alabama,AL,1,-86.902298,32.318231,0.97,1.18,7.09,13.70,...,4.09,6.18,2.51,1.33,100.83,8.89,5.91,8.89,31400.1046,1
2,2000-01-18,Alabama,AL,1,-86.902298,32.318231,1.80,1.59,9.91,6.50,...,2.75,6.29,3.87,2.28,100.82,13.94,11.96,13.94,31400.1046,1
3,2000-01-25,Alabama,AL,1,-86.902298,32.318231,0.00,2.20,2.15,10.89,...,3.71,7.61,4.35,2.16,100.53,-7.34,-4.39,-7.14,31400.1046,1
4,2000-02-01,Alabama,AL,1,-86.902298,32.318231,0.01,0.73,3.38,15.38,...,5.12,6.04,2.41,1.69,101.43,-1.58,-3.99,-1.55,31400.1046,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58235,2021-05-18,Wyoming,WY,56,-107.290284,43.075968,0.00,0.47,6.54,17.38,...,6.01,6.73,3.36,2.89,80.75,4.49,6.61,10.45,60197.7468,1
58236,2021-05-25,Wyoming,WY,56,-107.290284,43.075968,0.11,0.95,5.07,16.60,...,7.49,9.18,6.45,5.50,81.35,1.03,3.02,6.50,60197.7468,1
58237,2021-06-01,Wyoming,WY,56,-107.290284,43.075968,1.19,1.75,7.33,12.02,...,3.61,6.54,4.33,2.58,81.70,6.35,9.74,11.27,60197.7468,1
58238,2021-06-08,Wyoming,WY,56,-107.290284,43.075968,0.00,0.49,6.30,17.62,...,13.17,14.71,11.86,11.36,80.92,3.91,12.01,12.41,60197.7468,1


In [50]:
dfMerge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58240 entries, 0 to 58239
Data columns (total 26 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Week         58240 non-null  object 
 1   State        58240 non-null  object 
 2   Postal Code  58240 non-null  object 
 3   Fips         58240 non-null  int64  
 4   Longitude    58240 non-null  float64
 5   Latitude     58240 non-null  float64
 6   PRECTOT      58240 non-null  float64
 7   WS10M_MIN    58240 non-null  float64
 8   QV2M         58240 non-null  float64
 9   T2M_RANGE    58240 non-null  float64
 10  WS10M        58240 non-null  float64
 11  T2M          58240 non-null  float64
 12  WS50M_MIN    58240 non-null  float64
 13  T2M_MAX      58240 non-null  float64
 14  WS50M        58240 non-null  float64
 15  TS           58240 non-null  float64
 16  WS50M_RANGE  58240 non-null  float64
 17  WS50M_MAX    58240 non-null  float64
 18  WS10M_MAX    58240 non-null  float64
 19  WS10

In [51]:
dfMerge['Week'] = pd.to_datetime(dfMerge['Week'])
dfMerge

<ipython-input-51-77c61ad14a1c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfMerge['Week'] = pd.to_datetime(dfMerge['Week'])


,Week,State,Postal Code,Fips,Longitude,Latitude,PRECTOT,WS10M_MIN,QV2M,T2M_RANGE,...,WS50M_RANGE,WS50M_MAX,WS10M_MAX,WS10M_RANGE,PS,T2MDEW,T2M_MIN,T2MWET,Aland_SQMI,LEVEL
0,2000-01-04,Alabama,AL,1,-86.902298,32.318231,12.35,2.14,6.84,15.39,...,5.24,8.80,5.44,3.30,100.72,6.95,2.85,6.97,31400.1046,1
1,2000-01-11,Alabama,AL,1,-86.902298,32.318231,0.97,1.18,7.09,13.70,...,4.09,6.18,2.51,1.33,100.83,8.89,5.91,8.89,31400.1046,1
2,2000-01-18,Alabama,AL,1,-86.902298,32.318231,1.80,1.59,9.91,6.50,...,2.75,6.29,3.87,2.28,100.82,13.94,11.96,13.94,31400.1046,1
3,2000-01-25,Alabama,AL,1,-86.902298,32.318231,0.00,2.20,2.15,10.89,...,3.71,7.61,4.35,2.16,100.53,-7.34,-4.39,-7.14,31400.1046,1
4,2000-02-01,Alabama,AL,1,-86.902298,32.318231,0.01,0.73,3.38,15.38,...,5.12,6.04,2.41,1.69,101.43,-1.58,-3.99,-1.55,31400.1046,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58235,2021-05-18,Wyoming,WY,56,-107.290284,43.075968,0.00,0.47,6.54,17.38,...,6.01,6.73,3.36,2.89,80.75,4.49,6.61,10.45,60197.7468,1
58236,2021-05-25,Wyoming,WY,56,-107.290284,43.075968,0.11,0.95,5.07,16.60,...,7.49,9.18,6.45,5.50,81.35,1.03,3.02,6.50,60197.7468,1
58237,2021-06-01,Wyoming,WY,56,-107.290284,43.075968,1.19,1.75,7.33,12.02,...,3.61,6.54,4.33,2.58,81.70,6.35,9.74,11.27,60197.7468,1
58238,2021-06-08,Wyoming,WY,56,-107.290284,43.075968,0.00,0.49,6.30,17.62,...,13.17,14.71,11.86,11.36,80.92,3.91,12.01,12.41,60197.7468,1


In [52]:
dfMerge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58240 entries, 0 to 58239
Data columns (total 26 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Week         58240 non-null  datetime64[ns]
 1   State        58240 non-null  object        
 2   Postal Code  58240 non-null  object        
 3   Fips         58240 non-null  int64         
 4   Longitude    58240 non-null  float64       
 5   Latitude     58240 non-null  float64       
 6   PRECTOT      58240 non-null  float64       
 7   WS10M_MIN    58240 non-null  float64       
 8   QV2M         58240 non-null  float64       
 9   T2M_RANGE    58240 non-null  float64       
 10  WS10M        58240 non-null  float64       
 11  T2M          58240 non-null  float64       
 12  WS50M_MIN    58240 non-null  float64       
 13  T2M_MAX      58240 non-null  float64       
 14  WS50M        58240 non-null  float64       
 15  TS           58240 non-null  float64       
 16  WS50

In [53]:
dfMerge.describe()

,Fips,Longitude,Latitude,PRECTOT,WS10M_MIN,QV2M,T2M_RANGE,WS10M,T2M,WS50M_MIN,...,WS50M_RANGE,WS50M_MAX,WS10M_MAX,WS10M_RANGE,PS,T2MDEW,T2M_MIN,T2MWET,Aland_SQMI,LEVEL
count,58240.000000,58240.000000,58240.000000,58240.00000,58240.000000,58240.000000,58240.000000,58240.00000,58240.000000,58240.000000,...,58240.000000,58240.000000,58240.000000,58240.000000,58240.000000,58240.000000,58240.000000,58240.000000,58240.000000,58240.000000
mean,29.788462,-92.824719,39.153437,2.61711,1.835317,7.427130,10.906271,3.38326,11.378354,3.002745,...,4.462053,7.464751,4.936783,3.101474,94.982423,5.678410,6.155548,5.733265,41331.474327,0.480100
std,16.612622,19.244063,6.900434,5.87921,1.296068,4.742872,4.012301,1.75400,11.426394,2.065527,...,2.102334,2.762730,2.479797,1.844658,7.455071,10.760299,11.176480,10.680526,52615.590875,0.877662
min,1.000000,-155.665857,18.220833,0.00000,0.000000,0.070000,0.270000,0.48000,-41.340000,0.000000,...,0.360000,0.970000,0.810000,0.190000,69.880000,-44.690000,-42.900000,-41.320000,37.851000,0.000000
25%,16.750000,-102.009600,35.438381,0.01000,0.890000,3.460000,7.970000,2.05000,2.600000,1.360000,...,2.960000,5.450000,3.010000,1.740000,93.730000,-2.610000,-2.010000,-2.570000,17704.148050,0.000000
50%,29.500000,-89.093198,39.435516,0.25000,1.570000,6.140000,10.800000,2.96000,12.550000,2.640000,...,4.050000,6.970000,4.400000,2.650000,98.010000,5.740000,6.610000,5.760000,31831.051100,0.000000
75%,42.500000,-78.291302,43.220246,2.34000,2.470000,11.022500,13.720000,4.34000,20.950000,4.230000,...,5.520000,9.010000,6.350000,3.990000,100.040000,15.080000,15.440000,15.070000,49699.035700,1.000000
max,72.000000,-66.590149,63.588753,150.61000,13.460000,21.810000,28.420000,16.78000,37.740000,16.710000,...,22.130000,26.200000,20.060000,16.510000,103.850000,26.130000,29.790000,26.130000,353797.389000,4.000000


In [54]:
dfMerge.describe(include='all')

,Week,State,Postal Code,Fips,Longitude,Latitude,PRECTOT,WS10M_MIN,QV2M,T2M_RANGE,...,WS50M_RANGE,WS50M_MAX,WS10M_MAX,WS10M_RANGE,PS,T2MDEW,T2M_MIN,T2MWET,Aland_SQMI,LEVEL
count,58240,58240,58240,58240.000000,58240.000000,58240.000000,58240.00000,58240.000000,58240.000000,58240.000000,...,58240.000000,58240.000000,58240.000000,58240.000000,58240.000000,58240.000000,58240.000000,58240.000000,58240.000000,58240.000000
unique,1120,52,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2018-07-17 00:00:00,New Hampshire,NY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,52,1120,1120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,2000-01-04 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2021-06-15 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,29.788462,-92.824719,39.153437,2.61711,1.835317,7.427130,10.906271,...,4.462053,7.464751,4.936783,3.101474,94.982423,5.678410,6.155548,5.733265,41331.474327,0.480100
std,NaN,NaN,NaN,16.612622,19.244063,6.900434,5.87921,1.296068,4.742872,4.012301,...,2.102334,2.762730,2.479797,1.844658,7.455071,10.760299,11.176480,10.680526,52615.590875,0.877662
min,NaN,NaN,NaN,1.000000,-155.665857,18.220833,0.00000,0.000000,0.070000,0.270000,...,0.360000,0.970000,0.810000,0.190000,69.880000,-44.690000,-42.900000,-41.320000,37.851000,0.000000
25%,NaN,NaN,NaN,16.750000,-102.009600,35.438381,0.01000,0.890000,3.460000,7.970000,...,2.960000,5.450000,3.010000,1.740000,93.730000,-2.610000,-2.010000,-2.570000,17704.148050,0.000000


In [55]:
# Null detection

dfMerge.isnull().sum()

Week           0
State          0
Postal Code    0
Fips           0
Longitude      0
Latitude       0
PRECTOT        0
WS10M_MIN      0
QV2M           0
T2M_RANGE      0
WS10M          0
T2M            0
WS50M_MIN      0
T2M_MAX        0
WS50M          0
TS             0
WS50M_RANGE    0
WS50M_MAX      0
WS10M_MAX      0
WS10M_RANGE    0
PS             0
T2MDEW         0
T2M_MIN        0
T2MWET         0
Aland_SQMI     0
LEVEL          0
dtype: int64

In [56]:
# Duplicated detection

dfMerge.duplicated().sum()

0

In [57]:
# Outlier detection 

def detect_outliers(df, n, features):
    all_outliers_rows = []
    for col in features:
        Q1 = np.percentile(df[col], 25)
        Q3 = np.percentile(df[col], 75)
        IQR = Q3 - Q1
        IQR_Range = 1.5 * IQR
        col_outlier = df[(df[col] < Q1 - IQR_Range) | (df[col] > Q3 + IQR_Range)].index
        all_outliers_rows.extend(col_outlier)

    all_outliers_rows = Counter(all_outliers_rows)        
    multiple_outliers = list(k for k, v in all_outliers_rows.items() if v > n )
    return multiple_outliers   

Outliers_to_drop = detect_outliers(dfMerge, 3, dfMerge.columns[6:len(dfMerge.columns)-1])
print(len(Outliers_to_drop))

1448


In [58]:
dfMerge.loc[Outliers_to_drop] # Show the outliers rows

,Week,State,Postal Code,Fips,Longitude,Latitude,PRECTOT,WS10M_MIN,QV2M,T2M_RANGE,...,WS50M_RANGE,WS50M_MAX,WS10M_MAX,WS10M_RANGE,PS,T2MDEW,T2M_MIN,T2MWET,Aland_SQMI,LEVEL
1124,2000-02-01,Alaska,AK,2,-154.493062,63.588753,7.03,1.63,0.77,28.42,...,3.55,7.18,4.22,2.59,95.39,-23.36,-35.14,-23.35,353797.3890,0
2758,2009-12-08,Arizona,AZ,4,-111.093731,34.048928,7.71,0.83,3.27,10.79,...,13.76,15.04,9.45,8.62,82.93,-5.00,-8.74,-4.97,70428.3296,2
2771,2010-03-09,Arizona,AZ,4,-111.093731,34.048928,9.05,1.07,4.03,10.36,...,11.17,13.07,8.48,7.42,83.01,-2.04,-4.34,-2.02,70428.3296,1
3562,2003-11-18,Arkansas,AR,5,-91.831833,35.201050,36.09,1.64,11.06,10.11,...,12.73,15.68,10.60,8.96,98.76,15.13,11.76,15.13,32261.9976,0
3781,2008-01-29,Arkansas,AR,5,-91.831833,35.201050,12.28,2.35,6.75,25.83,...,14.16,18.99,14.28,11.93,99.18,5.56,-4.60,5.61,32261.9976,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50819,2008-01-15,Utah,UT,49,-111.093731,39.320980,0.11,3.65,1.65,15.26,...,8.30,14.36,11.82,8.16,76.29,-13.90,-20.00,-13.82,50945.1644,2
50859,2008-10-21,Utah,UT,49,-111.093731,39.320980,0.02,3.10,2.87,14.14,...,9.32,14.42,11.46,8.36,76.96,-7.52,-5.30,-7.35,50945.1644,0
50934,2010-03-30,Utah,UT,49,-111.093731,39.320980,0.00,3.94,3.48,15.27,...,9.12,16.46,12.48,8.55,75.52,-4.96,-1.00,-4.91,50945.1644,0
57560,2008-06-10,Wyoming,WY,56,-107.290284,43.075968,1.89,2.77,5.62,18.37,...,9.24,14.46,11.58,8.81,80.45,2.27,3.77,2.29,60197.7468,0


In [59]:
dfMerge = dfMerge.drop(Outliers_to_drop, axis = 0).reset_index(drop=True) # Drop outliers
dfMerge

,Week,State,Postal Code,Fips,Longitude,Latitude,PRECTOT,WS10M_MIN,QV2M,T2M_RANGE,...,WS50M_RANGE,WS50M_MAX,WS10M_MAX,WS10M_RANGE,PS,T2MDEW,T2M_MIN,T2MWET,Aland_SQMI,LEVEL
0,2000-01-04,Alabama,AL,1,-86.902298,32.318231,12.35,2.14,6.84,15.39,...,5.24,8.80,5.44,3.30,100.72,6.95,2.85,6.97,31400.1046,1
1,2000-01-11,Alabama,AL,1,-86.902298,32.318231,0.97,1.18,7.09,13.70,...,4.09,6.18,2.51,1.33,100.83,8.89,5.91,8.89,31400.1046,1
2,2000-01-18,Alabama,AL,1,-86.902298,32.318231,1.80,1.59,9.91,6.50,...,2.75,6.29,3.87,2.28,100.82,13.94,11.96,13.94,31400.1046,1
3,2000-01-25,Alabama,AL,1,-86.902298,32.318231,0.00,2.20,2.15,10.89,...,3.71,7.61,4.35,2.16,100.53,-7.34,-4.39,-7.14,31400.1046,1
4,2000-02-01,Alabama,AL,1,-86.902298,32.318231,0.01,0.73,3.38,15.38,...,5.12,6.04,2.41,1.69,101.43,-1.58,-3.99,-1.55,31400.1046,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56787,2021-05-11,Wyoming,WY,56,-107.290284,43.075968,0.70,2.00,3.90,10.46,...,4.09,7.08,6.11,4.11,81.57,-2.45,-2.92,0.02,60197.7468,2
56788,2021-05-18,Wyoming,WY,56,-107.290284,43.075968,0.00,0.47,6.54,17.38,...,6.01,6.73,3.36,2.89,80.75,4.49,6.61,10.45,60197.7468,1
56789,2021-05-25,Wyoming,WY,56,-107.290284,43.075968,0.11,0.95,5.07,16.60,...,7.49,9.18,6.45,5.50,81.35,1.03,3.02,6.50,60197.7468,1
56790,2021-06-01,Wyoming,WY,56,-107.290284,43.075968,1.19,1.75,7.33,12.02,...,3.61,6.54,4.33,2.58,81.70,6.35,9.74,11.27,60197.7468,1


In [ ]:
dfMerge.dtypes

In [60]:
dfMerge.to_csv(base_url + 'files/merge.csv')